<a href="https://colab.research.google.com/github/Nyota-Lab/movies/blob/Features/Creacion_y_Transformacion_de_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Creacion y Transformacion de Features
* Dado que nuestro problema de rendimiento no viene condicionado por la falta de features, es necesario crear nuevas o transformar las existentes



In [0]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

##Reescalamiento de los datos
* Es recomendable reescalar los datos para poder tenerlos de una forma mas compacta LAS REGRESIONES NO SE VEN AFECTADAS POR EL REESCALAMIENTO LAS CLASIFICACIONES SI LO SON, existen dos formas de hacer este reescalamiento de los datos:
 * Llevarlo a cero haciendo una resta del promedio de los valores que tienen las features
 * Dividirlo por su desviacion estandar

In [61]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#X = pd.read_csv(r'C:\Users\willi\Documents\Programacion\ML\x.csv')
path = ('/content/drive/My Drive/Peliculas/X.csv')
X = pd.read_csv('/content/drive/My Drive/Peliculas/X.csv')

* Como siempre volvemos a generar los conjuntos de Test y Train

In [0]:
y = X['worldwide_gross']
X = X.drop(['worldwide_gross','gross'], axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y)

* Usamos preprocessing para calcular la desviacion estandar y lo instanciamos

In [64]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

* scaler tiene algunos métodos pero dentro de estos podemos calcular la media y la desviacion estandar de cada uno de los features

In [65]:
scaler.mean_

array([3.32173071e+07, 2.00216971e+03, 2.12130186e+00, 1.08690771e+02,
       1.00827008e+04, 3.73667960e+07, 6.44580897e+00])

In [66]:
scaler.scale_

array([4.05797687e+07, 1.19323638e+01, 6.74618060e-01, 2.32546381e+01,
       1.81442411e+04, 6.68555370e+07, 1.07899204e+00])

* Observemos que actualmente los valores están en diferentes escalas y tiene muchas diferencias entre si

In [67]:
X_train.values

array([[1.5000000e+08, 2.0120000e+03, 1.8500000e+00, ..., 1.6184000e+04,
        1.5000000e+08, 5.8000000e+00],
       [7.1682975e+07, 2.0040000e+03, 1.8500000e+00, ..., 7.2730000e+03,
        6.0000000e+07, 6.5000000e+00],
       [2.3000000e+07, 1.9870000e+03, 2.3500000e+00, ..., 2.3550000e+03,
        2.3000000e+07, 2.8000000e+00],
       ...,
       [3.5000000e+07, 2.0060000e+03, 1.8500000e+00, ..., 2.4640000e+04,
        3.5000000e+07, 7.3000000e+00],
       [3.0000000e+07, 2.0040000e+03, 1.8500000e+00, ..., 2.0097000e+04,
        3.0000000e+07, 7.0000000e+00],
       [1.8000000e+07, 2.0050000e+03, 1.8500000e+00, ..., 1.6150000e+03,
        1.7000000e+07, 6.3000000e+00]])

* scaler tiene otro metofo que es transform el cual nos va a permitir realizar el escalamiento de los datos
* Observamos que nuestros datos ya pasan de estar en una escala muy diferente a encontrarse en valores decimales o muy cercanos a cero y entre sí

In [68]:
scaler.transform(X_train)

array([[ 2.87785507,  0.82383464, -0.40215624, ...,  0.33626643,
         1.68472514, -0.59852987],
       [ 0.94790259,  0.1533891 , -0.40215624, ..., -0.15485358,
         0.33853896,  0.05022376],
       [-0.25178328, -1.27130765,  0.33900387, ..., -0.42590377,
        -0.21489314, -3.37890256],
       ...,
       [ 0.04393058,  0.32100049, -0.40215624, ...,  0.80230962,
        -0.03540165,  0.79165647],
       [-0.07928352,  0.1533891 , -0.40215624, ...,  0.55192715,
        -0.11018977,  0.5136192 ],
       [-0.37499738,  0.2371948 , -0.40215624, ..., -0.46668806,
        -0.30463888, -0.13513442]])

* Tenemos quue tener en cuenta que tanto los datos de train como los de test deben estar escalados con el mismo scaler
vamos a crear dos nuevos grupos de datos

In [0]:
X_train_scaled, X_test_scaled = (scaler.transform(X_train), scaler.transform(X_test))

* Una vez tenemos nuestros datos normales y escalados es hora de probar si existe una mejora en el rendimiento de nuestro modelo Lasso

In [70]:
from sklearn.linear_model import Lasso
model = Lasso()
model_scaled = Lasso()

model.fit(X_train, y_train)
model_scaled.fit(X_train_scaled, y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [71]:
print(model.score(X_test, y_test))
print(model_scaled.score(X_test_scaled, y_test))

0.582437977847351
0.5824379819402636


* Como explicamos al principio el reescalamiento puede generar mejoras en el rendimiento de algunos modelos pero en las regresiones especificamente no tiene un impacto siginificativo

##Pipelines

* Podemos simplificar las transformaciones con pipelines de esta manera el podemos unir algunos de los pasos previos para realizar la estandarizacion esto resulta en un codigo mucho mas replicable

In [72]:
from sklearn.pipeline import make_pipeline
model_scaled = make_pipeline(StandardScaler(),
                            Lasso())
model_scaled.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('lasso',
                 Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=1000, normalize=False, positive=False,
                       precompute=False, random_state=None, selection='cyclic',
                       tol=0.0001, warm_start=False))],
         verbose=False)

* Comprobamos que nuestro pipeline funcione correctamente

In [73]:
print(model_scaled.score(X_test, y_test))

0.5824379819402636


##Creación de Features de forma automática
* sklearn tambien nos ofrece una forma de crear nuevas features a partir de las que tenemos de forma automática
* Para entender como funciona vamos a crear una matriz y le aplicaremos la creación de features

In [74]:
A = np.arange(6).reshape(3, 2)
A

array([[0, 1],
       [2, 3],
       [4, 5]])

* Importamos PolynomialFeatures y lo instanciamos
* Este metodo tiene un parámetro que es el grado de los polinomios

In [75]:
from sklearn.preprocessing import PolynomialFeatures
transformer = PolynomialFeatures(2)
#regularmente el metodo trasnform se usa solo una vez, para evitar escribir lineas de código
#podemos usarlo de la siguiente manera
transformer.fit_transform(A)

array([[ 1.,  0.,  1.,  0.,  0.,  1.],
       [ 1.,  2.,  3.,  4.,  6.,  9.],
       [ 1.,  4.,  5., 16., 20., 25.]])

* Ahora vamos a aplicar la creacion de features para nuestro dataframe en particular, vemos que hasta el momento tenemos 7

In [76]:
X.shape

(4104, 7)

* Aplicamos el método transformer
* Este transformador genera las combinaciones posibles entre features entre mas features iniciales tengamos, mas se van a generar con el metodo transform

In [79]:
transformer = PolynomialFeatures(2)
X4 = transformer.fit_transform(X)
print(X4.shape)
from sklearn.model_selection import train_test_split
X4_train, X4_test, y4_train, y4_test = train_test_split(X4,y)
print(X4_train.shape)

(4104, 36)
(3078, 36)


* Vamos a entrenar nuestro modelo con las nuevas features
* Verificamos que nuestro rendimiento baja mucho, esto se debe a que aumentamos demasiadas features

In [80]:
model_poly = Lasso()
model_poly.fit(X4_train,y4_train)
model_poly.score(X4_test,y4_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.6922630498782069e+19, tolerance: 9790433278244940.0
  positive)


0.348316754699998

In [83]:
X4_train.shape

(3078, 36)

##Creacion a partir de variables categoricas (One Hot Encoding)
* Es posible crear features de orden categórica con el fin de poder hacer un análisis de las mismas
asignar un numero a cada valor en un orden (1,2,3,4,...) es que la distancia entre ellos puede afectar el rendimiento
en este caso se usa un Encoding one-hot, el cual crea columnas y las rellena con 1 y 0 dependiendo del valor


In [0]:
path = ('/content/drive/My Drive/Peliculas/movies_obj.csv')
movies_obj = pd.read_csv('/content/drive/My Drive/Peliculas/movies_obj.csv')

* Primero miramos la cantidad de valores unicos que tiene cada una de las features, nunique nos ayuda a contar los valores unicos y sort_values()los ordena 
* Hay que tener en cuenta que se puede aumentar demasiadas features utilizando éste método


In [87]:
movies_obj.apply(pd.Series.nunique).sort_values()

color                2
content_rating      18
language            47
country             65
genres             914
actor_1_name      2097
director_name     2398
actor_2_name      3032
actor_3_name      3521
plot_keywords     4760
movie_title       4917
dtype: int64

##Encoding Binario
* Otra opcion es hacer un encoding binario es decir convertir la categoría en un número este numero se convierte en un binario y creamos columnas de acuerdo al mayor numero
* Este al no ser un metodo propio de sklearn es necesario cargar otra libreria

In [88]:
!pip install category_encoders

     |████████████████████████████████| 102kB 4.1MB/s 


* Tenemos un dataframe con solo 2 columnas que son las cuales nos van a servir para esta transformacion y lo cargamos

In [0]:
path =('/content/drive/My Drive/Peliculas/categoricals.csv')
categoricals = pd.read_csv('/content/drive/My Drive/Peliculas/categoricals.csv')

* Conocemos que existen algunos valores faltantes en nuestro dataframe por lo que vamos a reemplazarlos por 0

In [0]:
categoricals = categoricals.reset_index(drop=True).fillna(0)

* Una cez arreglamos los datos faltantes en nuestro dataframe concatenamos los dos

In [0]:
X_binenc = pd.concat((X,categoricals),axis=1)

* Importamos category_encoders e instanciamos el mismo usando BinaryEncoder y seleccionando las columnas a usar

In [0]:
import category_encoders as ce
encoder = ce.BinaryEncoder(cols=['actor_1_name','director_name'])

In [99]:
X_binenc = encoder.fit_transform(X_binenc)
print(X_binenc.shape)
print(y.shape)

(4104, 32)
(4104,)


* Ahora generamos nuestros conjuntos de prueba y entrenamiento

In [0]:
Xb_train, Xb_test, y_train, y_test = train_test_split(X_binenc,y)

* Vamos a generar tambien unos conjuntos de comprobancion para ver la diferencia en el rendimiento de los dos dataframes

In [0]:
X_train, X_test = (Xb_train[X.columns], Xb_test[X.columns])

* Entrenamos los modelos con el binary_encoding y el de control

In [0]:
model_binenc = Lasso()
model = Lasso()


In [107]:
model_binenc.fit(Xb_train,y_train)
model.fit(X_train,y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [108]:
print(model_binenc.score(Xb_train,y_train))
print(model.score(X_train,y_train))

0.5827002282133342
0.5705199150894189
